# TE Regressions with ZKML

## Create and export model via TF -> TFLite

In [ ]:
# check if notebook is in colab
try:
    # install ezkl
    import google.colab
    import subprocess
    import sys

    subprocess.check_call([sys.executable, "-m", "pip", "install", "ezkl"])
    subprocess.check_call([sys.executable, "-m", "pip", "install", "onnx"])
    subprocess.check_call([sys.executable, "-m", "pip", "install", "sk2torch"])

# rely on local installation of ezkl if the notebook is not in colab
except:
    pass


# here we create and (potentially train a model)

# make sure you have the dependencies required here already installed
import json
import sys
import numpy as np
from sklearn import datasets
import sk2torch
import torch
import ezkl
import os
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
sys.path.insert(0, "../../..")
from zkml.python.converter import Converter

CKPT_PATH = os.path.join(os.getcwd(), "svm")
TFLITE_PATH = os.path.join(os.getcwd(), "svm.tflite")
OUT_MODEL_PATH = os.path.join(os.getcwd(), "svm.msgpack")
OUT_CONFIG_PATH = os.path.join(os.getcwd(), "svm_config.json")
OUT_INPUT_NPY_PATH = os.path.join(os.getcwd(), "svm_input.npy")
OUT_INPUT_MSG_PATH = os.path.join(os.getcwd(), "svm_input.msgpack")
INPUT_CONVERTER_PATH = os.path.join(
    os.getcwd(), "../../src/zkml/python/input_converter.py"
)

X = np.array([[1, 1], [1, 2], [2, 2], [2, 3], [3, 3], [4, 4], [6, 8]])
y = np.dot(X, np.array([1, 2])) + 3  # y = 1 * x_0 + 2 * x_1 + 3

# Define and train the model
# This is a simple NN regression model. It isn't possible to implement a random forest regressor in Keras / TFLite.
model = keras.Sequential([
    layers.Dense(64, activation='relu', input_shape=(2,)),
    layers.Dense(64, activation='relu'),
    layers.Dense(1)
])

model.compile(optimizer='adam', loss='mse')
model.fit(X, y, epochs=10, batch_size=1)

# Convert the model to TensorFlow Lite
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

# Save the TFLite model
with open(TFLITE_PATH, 'wb') as f:
    f.write(tflite_model)

## Convert model to msgpack using ZKML

In [ ]:
converter = Converter(model_path=TFLITE_PATH, expose_output=False, commit=False, scale_factor=1, k=9, num_cols=2, num_randoms=6, use_selectors=True)

model_packed, config_packed = converter.to_msgpack(
  start_layer=0,
  end_layer=10000,
)
if model_packed is None:
  raise Exception('Failed to convert model')
with open(OUT_MODEL_PATH, 'wb') as f:
  f.write(model_packed)
with open(OUT_CONFIG_PATH, 'wb') as f:
  f.write(config_packed)

## Create msgpack from input

In [ ]:
with open (OUT_INPUT_NPY_PATH, 'wb') as f:
    np.save(f, np.array(X[0]).reshape(-1, 1))

!python {INPUT_CONVERTER_PATH} --model_config {OUT_MODEL_PATH} --inputs {OUT_INPUT_NPY_PATH} --output {OUT_INPUT_MSG_PATH}


## Generate the proof

In [ ]:
!../../src/zkml/target/release/time_circuit {OUT_MODEL_PATH} {OUT_INPUT_MSG_PATH} kzg
